In [1]:
from langchain_openai import ChatOpenAI
from typing import List
from typing import TypedDict, Annotated, Sequence
import os, operator, json
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents.agent_types import AgentType
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
import time


OPENAI_API_KEY="sk-vppxxxLaft8GdCx4PxxxXT3BlxxFJWGVSIWWCtErf3tWZgz5x"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [2]:
contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "05/31/2025",
  "Contract_amount": "$90,000.00"}"""

In [3]:
llm = ChatOpenAI(model="gpt-4-1106-preview")

# Create Agent Using Customized Function

In [10]:
from datetime import datetime
def getting_yearly_amount(contract):
#def getting_yearly_amount(contract=f"{contract}"):
    try:
        contract = json.loads(contract)
    except json.JSONDecodeError as e:
        print('Failed to decode coefficients:', e)
        return None
    # Extracting required information from the contract
    term_start_date = datetime.strptime(contract['Term_start_date'], '%m/%d/%Y')
    term_end_date = datetime.strptime(contract['Term_end_date'], '%m/%d/%Y')
    contract_amount = float(contract['Contract_amount'].replace('$', '').replace(',', ''))

    # Calculating the total number of months for the contract period
    total_months = (term_end_date.year - term_start_date.year) * 12 + (term_end_date.month - term_start_date.month + 1)

    # Finding the total number of years
    total_years = term_end_date.year - term_start_date.year + 1

    # Getting the number of months in each year
    months_per_year = {}
    for year in range(term_start_date.year, term_end_date.year + 1):
        if year == term_start_date.year:
            months_per_year[year] = 12 - term_start_date.month + 1
        elif year == term_end_date.year:
            months_per_year[year] = term_end_date.month
        else:
            months_per_year[year] = 12

    # Calculating the amount corresponds to each month
    amount_per_month = contract_amount / total_months

    # Calculating the amount corresponds to each year
    amount_per_year = {year: months * amount_per_month for year, months in months_per_year.items()}

    return amount_per_year

equation_tool = Tool(name='Yearly Amount Estimation', func=getting_yearly_amount, description="Useful for data provided in the json format and estimating amount assciated for each year.")

tools = [equation_tool]

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3, input_key="input", output_key="output",return_messages=True) 

agent_parameters = {
    'agent':'chat-conversational-react-description',
    'tools':tools,
    'llm':llm,
    'verbose':True,
    'max_iterations':4,
    'return_intermediate_steps':True,
    'early_stopping_method':'generate',
  
    'memory':memory
}

agent = initialize_agent(**agent_parameters)


In [11]:
contract = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "05/31/2025",
  "Contract_amount": "$90,000.00"}"""

In [12]:
%time
output= agent.invoke({"input":f"get each year amount from the given contract {contract} "})


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 17.9 µs


> Entering new AgentExecutor chain...
```json
{
    "action": "Yearly Amount Estimation",
    "action_input": "{\"Sno\": 1, \"Contract_Number\": \"C887/36\", \"Contract_Date\": \"11/1/2023\", \"Term_start_date\": \"12/1/2023\", \"Term_end_date\": \"05/31/2025\", \"Contract_amount\": \"$90,000.00\"}"
}
```
Observation: {2023: 5000.0, 2024: 60000.0, 2025: 25000.0}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The estimated contract amount allocated for each year is as follows: For the year 2023, the amount is $5,000.00; for the year 2024, it's $60,000.00; and for the year 2025, the amount is $25,000.00."
}
```

> Finished chain.


In [13]:
result=output['intermediate_steps'][-1][-1]
result

{2023: 5000.0, 2024: 60000.0, 2025: 25000.0}

# Creating an Agent Using Standard Tool and Prompt

In [14]:
problem_chain = LLMMathChain.from_llm(llm=llm)

math_tool = Tool.from_function(name="Calculator",func=problem_chain.run,description="Useful for when you need to answer questions \
about math. This tool is only for math questions and nothing else. Only input\
math expressions.")

agent = initialize_agent(
    tools=[math_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

In [15]:

contract={'Sno': 1, 'Contract_Number': 'C887/36', 'Contract_Date': '11/1/2023', 'Term_start_date': '12/1/2023', 'Term_end_date': '05/31/2024', 'Contract_amount': '$90,000.00'},

instruction=f"""
From this dictionay {contract} extract the value of each key and perform the following steps
1. Get the total number of months for the given contract period. The beginning of the contract is Term_start_date and the end of the contract is Term_end_date. Include both the initial and final months in calculating the total number of months.
2. Find the total number of years in this contract. The beginning of the contract is Term_start_date and the end of the contract is Term_end_date. Include both the initial and final years in calculating the total number of years.
3. Get the number of months in each year. For example, if the Term_start_date is 11/1/2023 and Term_end_date is 8/1/2025, then there will be 2, 12 and 8 months in 2023, 2024 and 2025, respectively
4. Get the amount corresponds to each month. This will be Contract_amount divided by total number of months for the given contract period.
5. Get the amount corresponds to each year. This will be the multiplication of the amount corresponds to each month and the number of months in each year.
Provide the final result in the json format where year will be key and the amount corresponds to each year will be value. Don't put addional text in the final output.
"""

In [16]:
output=agent.invoke({'input':instruction})
output['output']

'{\n  "2023": 15000,\n  "2024": 75000\n}'

# Create an agent using python repl

In [20]:
from langchain import hub
from langchain_experimental.tools import PythonREPLTool
tools = [PythonREPLTool()]


In [21]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
User are encouraged to provide query with json data in this format.
{'Sno': 1,
  'Contract_Number': 'C887/36',
  'Contract_Date': '11/1/2023',
  'Term_start_date': '12/1/202323',
  'Term_end_date': '05/31/2024',
  'Contract_amount': '$90,000.00'}
If the user provide the json fomated input in the query, follow the following steps for to generate the final result.
1. Get the total number of months for the given contract period. The beginning of the contract is Term_start_date and the end of the contract is Term_end_date. Include both the initial and final months in calculating the total number of months.
2. Find the total number of years in this contract. The beginning of the contract is Term_start_date and the end of the contract is Term_end_date. Include both the initial and final years in calculating the total number of years.
3. Get the number of months in each year. For example, if the Term_start_date is 11/1/2023 and Term_end_date is 8/1/2025, then there will be 2, 12 and 8 months in 2023, 2024 and 2025, respectively
4. Get the amount corresponds to each month. This will be Contract_amount divided by total number of months for the given contract period.
5. Get the amount corresponds to each year. This will be the multiplication of the amount corresponds to each month and the number of months in each year.
Provide the final result in the json format where year will be a key and the amount corresponds to each year will be the value. Don't put addional text in the final output.
"""


In [22]:
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [23]:
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [24]:
query={'Sno': 1,
  'Contract_Number': 'C887/36',
  'Contract_Date': '11/1/2023',
  'Term_start_date': '12/1/202323',
  'Term_end_date': '05/31/2024',
  'Contract_amount': '$90,000.00'}


In [25]:

result=agent_executor.invoke({"input": query})

result['output']



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `from datetime import datetime

# Input data
contract_data = {'Sno': 1, 'Contract_Number': 'C887/36', 'Contract_Date': '11/1/2023', 'Term_start_date': '12/1/202323', 'Term_end_date': '05/31/2024', 'Contract_amount': '$90,000.00'}

# Correct the Term_start_date
contract_data['Term_start_date'] = '12/1/2023'

# Convert dates to datetime objects
start_date = datetime.strptime(contract_data['Term_start_date'], '%m/%d/%Y')
end_date = datetime.strptime(contract_data['Term_end_date'], '%m/%d/%Y')

# Calculate the total number of months
total_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1
print(total_months)`


6

Invoking: `Python_REPL` with `# Calculate the total number of years
start_year = start_date.year
end_year = end_date.year
total_years = end_year - start_year + 1
print(total_years)`


2

Invoking: `Python_REPL` with `# Calculate the number of months in each year
months_in_years = {}
for year in range(start_year, e

'```json\n{\n  "2023": 15000.0,\n  "2024": 75000.0\n}\n```'

In [26]:
result['output']

'```json\n{\n  "2023": 15000.0,\n  "2024": 75000.0\n}\n```'

In [30]:
# check this out: https://pub.towardsai.net/use-langchain-to-extract-a-dictionary-from-text-3fca094d85d
# from langchain.output_parsers import ResponseSchema
# from langchain.output_parsers import StructuredOutputParser

# year_schema = ResponseSchema(name="year",
#                              description="year covered in contract")

# amount_schema = ResponseSchema(name="amount",
#                              description="amount covered in each year")

# response_schemas = [year_schema, amount_schema]
                        

# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# output_dict = output_parser.parse(result['output'])

## Using Standard Tool

In [61]:
from langchain.agents import tool
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


OPENAI_API_KEY="sk-xxxEQr0Lxxx8GdCx4PrXT3BxxkFJWGVSIWWCtErf3tWZgz5x"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)


In [62]:
@tool
def getting_yearly_amount(contract) -> dict:
    """ Useful for doing mathematical calculation"""
    # try:
    #     contract = json.loads(contract)
    # except json.JSONDecodeError as e:
    #     print('Failed to decode coefficients:', e)
    #     return None
    # Extracting required information from the contract
    term_start_date = datetime.strptime(contract['Term_start_date'], '%m/%d/%Y')
    term_end_date = datetime.strptime(contract['Term_end_date'], '%m/%d/%Y')
    contract_amount = float(contract['Contract_amount'].replace('$', '').replace(',', ''))

    # Calculating the total number of months for the contract period
    total_months = (term_end_date.year - term_start_date.year) * 12 + (term_end_date.month - term_start_date.month + 1)

    # Finding the total number of years
    total_years = term_end_date.year - term_start_date.year + 1

    # Getting the number of months in each year
    months_per_year = {}
    for year in range(term_start_date.year, term_end_date.year + 1):
        if year == term_start_date.year:
            months_per_year[year] = 12 - term_start_date.month + 1
        elif year == term_end_date.year:
            months_per_year[year] = term_end_date.month
        else:
            months_per_year[year] = 12

    # Calculating the amount corresponds to each month
    amount_per_month = contract_amount / total_months

    # Calculating the amount corresponds to each year
    amount_per_year = {year: months * amount_per_month for year, months in months_per_year.items()}

    return amount_per_year


In [63]:
tools = [getting_yearly_amount]

extract_messages = lambda x: x["messages"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)


from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "messages": extract_messages, 
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [64]:
contract={'Sno': 1,
  'Contract_Number': 'C887/36',
  'Contract_Date': '11/1/2023',
  'Term_start_date': '12/1/2023',
  'Term_end_date': '05/31/2024',
  'Contract_amount': '$90,000.00'}


In [65]:
result=agent_executor.invoke({'input':contract, 'messages':[], 'agent_scratchpad':[]})



> Entering new AgentExecutor chain...

Invoking: `getting_yearly_amount` with `{'contract': {'Contract_Number': 'C887/36', 'Contract_Date': '11/1/2023', 'Term_start_date': '12/1/2023', 'Term_end_date': '05/31/2024', 'Contract_amount': '$90,000.00'}}`


{2023: 15000.0, 2024: 75000.0}The contract with the number C887/36, which has a total contract amount of $90,000.00, is allocated as follows:

- For the year 2023, the amount is $15,000.00.
- For the year 2024, the amount is $75,000.00.

> Finished chain.


In [66]:
result

{'input': {'Sno': 1,
  'Contract_Number': 'C887/36',
  'Contract_Date': '11/1/2023',
  'Term_start_date': '12/1/2023',
  'Term_end_date': '05/31/2024',
  'Contract_amount': '$90,000.00'},
 'messages': [],
 'agent_scratchpad': [],
 'output': 'The contract with the number C887/36, which has a total contract amount of $90,000.00, is allocated as follows:\n\n- For the year 2023, the amount is $15,000.00.\n- For the year 2024, the amount is $75,000.00.'}

In [67]:
print(result['output'])

The contract with the number C887/36, which has a total contract amount of $90,000.00, is allocated as follows:

- For the year 2023, the amount is $15,000.00.
- For the year 2024, the amount is $75,000.00.
